In [1]:
import pandas as pd
from pycaret.classification import *
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import os

In [2]:
#load data
os.chdir('../..')
df=pd.read_csv('./data/csv/graph-features-reduced.csv')
#drop node and cluster columns
df.drop(['node','cluster'],axis=1,inplace=True)
#rename label values botnet to 1 and normal to zero
df['label'].replace({'botnet':1,'normal':0},inplace=True)
#train logistic regression model with all numerical features
classif = setup(data = df, target = 'label', session_id=123, train_size=0.7,numeric_features=['ID','OD','IDW','ODW','LCC','AC','BC'],silent=True)

,Description,Value
0,session_id,123
1,Target,label
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(18693, 8)"
5,Missing Values,False
6,Numeric Features,7
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [3]:
#add balanced accuracy metric
add_metric('balanced_accuracy', 'Balanced Accuracy', balanced_accuracy_score, greater_is_better=True)

#Add specificity metric
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel() #ravel flattens the array
    specificity = tn / (tn+fp)
    return specificity
    
add_metric('specificity', 'Specificity', specificity_score, greater_is_better=True)



Name                                                    Specificity
Display Name                                            Specificity
Score Function       <function specificity_score at 0x7f9262b9d160>
Scorer                               make_scorer(specificity_score)
Target                                                         pred
Args                                                             {}
Greater is Better                                              True
Multiclass                                                     True
Custom                                                         True
Name: specificity, dtype: object

In [8]:
#logistic regression without penalization
lr = create_model('lr',penalty='none',class_weight='balanced')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balanced Accuracy,Specificity
Fold,,,,,,,,,
0,0.1230,0.5547,0.5000,0.0009,0.0017,-0.0013,-0.0449,0.3112,0.1224
1,0.1131,0.5471,0.5000,0.0009,0.0017,-0.0013,-0.0478,0.3062,0.1125
2,0.2162,0.5126,0.5000,0.0010,0.0019,-0.0011,-0.0270,0.3579,0.2158
3,0.1459,0.5134,0.5000,0.0009,0.0018,-0.0013,-0.0392,0.3227,0.1454
4,0.0940,0.9954,1.0000,0.0017,0.0034,0.0003,0.0125,0.5463,0.0926
5,0.0107,0.5555,1.0000,0.0008,0.0015,0.0000,0.0028,0.5050,0.0099
6,0.1789,0.9923,1.0000,0.0009,0.0019,0.0003,0.0129,0.5891,0.1783
7,0.1476,0.9885,1.0000,0.0009,0.0018,0.0003,0.0115,0.5735,0.1469
8,0.1628,0.9901,1.0000,0.0009,0.0018,0.0003,0.0122,0.5811,0.1622


In [9]:
#Save logistic regression average and stdev metrics
lr_results = pull()
#Keep Recall, Balanced Accuracy and specificity columns only
lr_results = lr_results[['Recall','Balanced Accuracy','Specificity']]
#Save dataframe rows called Mean and Std
lr_results = lr_results.loc[['Mean','Std']]
lr_results

,Recall,Balanced Accuracy,Specificity
Fold,,,
Mean,0.8000,0.4599,0.1198
Std,0.2449,0.1144,0.0634
